In [36]:
import pandas as pd

# Excel dosyasını okuma
file_path = "/Users/canyonder/Desktop/tam_listeler/empty_rows.xlsx"
df = pd.read_excel(file_path, engine='openpyxl')


In [38]:
df


,UnitId,UnitName,CityName,DistrictName,Old Latitude,Old Longitude,Corrected Latitude,Corrected Longitude
0,2DACAAC9-1446-48C1-A5DB-FA49B1E44737,ÖZEL BİTLİS DİDEBAN DİŞ KLİNİĞİ,Bitlis,Merkez,38.15135,40.97236,NaN,NaN
1,6303CADF-F11B-4436-97F0-055719687151,ÖNDER ÇELİK MUAYENEHANESİ,Uşak,Merkez,36.77134,28.80093,NaN,NaN
2,866FD17D-AAAB-455D-99C6-0AE15C533A61,GÜÇLÜ BİTGİN MUAYENEHANESİ,Uşak,Merkez,39.42047,-76.32516,NaN,NaN
3,A365B09A-A142-497A-B040-13138A88EBD4,ÖZEL KEMALÖZ ÖZTAN HASTANESİ YATAN HASTA ECZANESİ,Uşak,Merkez,40.83010,29.33200,NaN,NaN
4,E6AF1D26-8E57-4CDD-97CD-14503EA41899,ÖZDEMİR KOÇYİĞİT MUAYENEHANESİ,Uşak,Merkez,41.02511,28.87247,NaN,NaN
...,...,...,...,...,...,...,...,...
1040,FD9ACE01-698E-4E5A-AA95-2BF8A9122530,ELİF ECZANESİ,Afyonkarahisar,Merkez,39.40337,-76.34796,NaN,NaN
1041,23FCE139-C980-444C-AB42-7BF95A4CE3EF,ÖMER KANAT KÖKSAL MUAYENEHANESİ,Afyonkarahisar,Merkez,38.96687,-76.92996,NaN,NaN
1042,E0206767-8319-4D48-B177-7E12375BA28B,NADİRE TUNCER MUAYENEHANESİ,Afyonkarahisar,AFYON MERKEZ,39.12167,-76.72356,NaN,NaN
1043,83040A42-E47B-4FCB-BB70-75D5E9DE43D5,PAZARYERİ ÇİĞDEM ECZANESİ,Bilecik,Pazaryeri,39.16847,-76.66116,NaN,NaN


In [44]:
import pandas as pd
import requests
import time

# OpenCage API Anahtarınızı Buraya Girin
API_KEY = '546de7c67b264335badb237677413533'

# Dosya Yolları
input_file_path = "/Users/canyonder/Desktop/tam_listeler/empty_rows.xlsx"  # Giriş dosyan
output_file_path = "/Users/canyonder/Desktop/tam_listeler/sehir_kontrol_sonuc.xlsx"  # Çıkış dosyan

# Excel dosyasını oku ve sütun adlarını temizle
df = pd.read_excel(input_file_path)
df.columns = df.columns.str.strip()  # 🛠 Sütun isimlerindeki boşlukları kaldır

# OpenCage Reverse Geocoding Fonksiyonu
def get_city_name(lat, lon):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={lat}+{lon}&key={API_KEY}&language=tr"
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"API hatası: {response.status_code}")
        return None
    
    data = response.json()
    if data["total_results"] == 0:
        return None

    # Şehir bilgisini bulma
    result = data["results"][0]
    components = result.get("components", {})
    city = components.get("city") or components.get("province") or components.get("state")
    if not city:
        city = components.get("town") or components.get("region") or components.get("village")
    
    return city

# Satırları işle
for index, row in df.iterrows():
    lat, lon = row["Old Latitude"], row["Old Longitude"]  # ✅ Sütun isimleri artık hatasız
    
    if pd.isna(lat) or pd.isna(lon):
        continue
    
    api_city = get_city_name(lat, lon)
    excel_city = row["CityName"]

    # Ekranda anlık durumu yazdır
    print(f"Seçilen şehir: {excel_city}")
    print(f"Row {index+1}:")
    print(f"Checked: {row['UnitId']} - {row['UnitName']}  - {excel_city}")

    if api_city:
        print(f"Found: {api_city}")
        if api_city.lower() == excel_city.lower():
            print("Match: Correct")
            df.at[index, "City_Match"] = "✅ Eşleşti"
        else:
            print(f"Match: Incorrect (API: {api_city})")
            df.at[index, "City_Match"] = f"❌ API: {api_city}"
    else:
        print("Found: Şehir Bulunamadı")
        df.at[index, "City_Match"] = "❌ API: Şehir Bulunamadı"
    
    print("-" * 50)  # Satırları ayırmak için
    time.sleep(1)

# Sonuç dosyasını kaydet
df.to_excel(output_file_path, index=False)

# Tamamlandığında son bir bilgilendirme
print("İşlem tamamlandı! Sonuç dosyanız kaydedildi:", output_file_path)


Seçilen şehir: Bitlis
Row 1:
Checked: 2DACAAC9-1446-48C1-A5DB-FA49B1E44737 - ÖZEL BİTLİS DİDEBAN DİŞ KLİNİĞİ  - Bitlis
Found: Diyarbakır
Match: Incorrect (API: Diyarbakır)
--------------------------------------------------
Seçilen şehir: Uşak
Row 2:
Checked: 6303CADF-F11B-4436-97F0-055719687151 - ÖNDER ÇELİK MUAYENEHANESİ  - Uşak
Found: Muğla
Match: Incorrect (API: Muğla)
--------------------------------------------------
Seçilen şehir: Uşak
Row 3:
Checked: 866FD17D-AAAB-455D-99C6-0AE15C533A61 - GÜÇLÜ BİTGİN MUAYENEHANESİ  - Uşak
Found: Maryland
Match: Incorrect (API: Maryland)
--------------------------------------------------
Seçilen şehir: Uşak
Row 4:
Checked: A365B09A-A142-497A-B040-13138A88EBD4 - ÖZEL KEMALÖZ ÖZTAN HASTANESİ YATAN HASTA ECZANESİ  - Uşak
Found: İstanbul
Match: Incorrect (API: İstanbul)
--------------------------------------------------
Seçilen şehir: Uşak
Row 5:
Checked: E6AF1D26-8E57-4CDD-97CD-14503EA41899 - ÖZDEMİR KOÇYİĞİT MUAYENEHANESİ  - Uşak
Found: İstanbul
M